In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import math
import random as rd
import glob

In [2]:
allFiles = glob.glob("data/*_va3.csv")
df = pd.DataFrame()
list_ = []

for file_ in allFiles:
    df = pd.read_csv(file_, index_col=None, header=0, sep=',')
    list_.append(df)
    
df = pd.concat(list_, sort = True)
df.head()

,1,10,11,12,13,14,15,16,17,18,...,30,31,32,4,5,6,7,8,9,Phase
0,0.000194,-0.000148,0.000118,-0.000033,0.000008,-0.000015,-0.000005,0.000025,-0.000019,0.000003,...,0.000031,0.000016,0.000025,-0.000206,0.000130,-0.000048,0.000120,-0.000057,-0.000082,D
1,0.000254,-0.000109,0.000053,-0.000025,0.000022,-0.000033,-0.000005,0.000021,-0.000030,0.000003,...,0.000036,0.000037,0.000028,-0.000151,0.000049,-0.000040,0.000168,-0.000143,-0.000094,D
2,0.000276,-0.000053,-0.000033,-0.000010,0.000006,0.000004,0.000002,0.000021,-0.000027,0.000004,...,0.000034,0.000016,0.000027,-0.000070,-0.000056,-0.000024,0.000194,-0.000198,-0.000093,D
3,0.000308,-0.000023,-0.000079,-0.000002,0.000010,-0.000002,0.000001,0.000014,-0.000021,0.000003,...,0.000025,0.000022,0.000018,-0.000029,-0.000120,-0.000016,0.000230,-0.000253,-0.000098,D
4,0.000294,-0.000004,-0.000109,0.000002,-0.000004,0.000006,0.000003,0.000006,-0.000013,0.000001,...,0.000014,0.000010,0.000011,-0.000010,-0.000161,-0.000012,0.000218,-0.000281,-0.000095,D


In [3]:
len(df)

9873

In [4]:
y = df['Phase'].values.tolist()
#type(y); y.describe(); y.head(3); y.shape; y.unique() # (165632, )
dic = {'D':'Descanso', 'P':'Preparation', 'S':'Stroke', 'H':'Hold', 'R':'Retraction'}
y = [dic.get(n, n) for n in y]

In [5]:
X = df.drop(labels=['Phase'], axis=1, inplace = True)
X = df.astype(float)
X = df.values
X

array([[ 1.93940e-04, -1.48040e-04,  1.18210e-04, ...,  1.19730e-04,
        -5.71800e-05, -8.19000e-05],
       [ 2.54420e-04, -1.08820e-04,  5.33600e-05, ...,  1.68180e-04,
        -1.42560e-04, -9.39600e-05],
       [ 2.76360e-04, -5.26300e-05, -3.34800e-05, ...,  1.94360e-04,
        -1.97630e-04, -9.29200e-05],
       ...,
       [ 3.07408e-03,  4.13420e-04, -1.13780e-04, ...,  2.78558e-03,
         5.03483e-03, -6.06410e-04],
       [ 3.29680e-03,  5.79910e-04, -3.80930e-04, ...,  2.94716e-03,
         5.38513e-03, -6.51760e-04],
       [ 2.04320e-04,  6.44270e-04, -4.80730e-04, ...,  2.28860e-04,
         3.19000e-06,  6.14900e-05]])

In [6]:
class K_Means:
    def __init__(self, k=5, tol=0.001, max_iter=300):
        self.k = k
        self.tol = tol
        self.max_iter = max_iter

    def fit(self,data):

        self.centroids = {}

        for i in range(self.k):
            self.centroids[i] = data[i]

        for i in range(self.max_iter):
            self.classifications = {}

            for i in range(self.k):
                self.classifications[i] = []

            for featureset in data:
                distances = [np.linalg.norm(featureset-self.centroids[centroid]) for centroid in self.centroids]
                classification = distances.index(min(distances))
                self.classifications[classification].append(featureset)

            prev_centroids = dict(self.centroids)

            for classification in self.classifications:
                self.centroids[classification] = np.average(self.classifications[classification],axis=0)

            optimized = True

            for c in self.centroids:
                original_centroid = prev_centroids[c]
                current_centroid = self.centroids[c]
                if np.sum((current_centroid-original_centroid)/original_centroid*100.0) > self.tol:
                    #print(np.sum((current_centroid-original_centroid)/original_centroid*100.0))
                    optimized = False

            if optimized:
                break

    def predict(self,data):
        distances = [np.linalg.norm(data-self.centroids[centroid]) for centroid in self.centroids]
        classification = distances.index(min(distances))
        return classification

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=50)

print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

4936
4937
4936
4937


In [8]:
clf = K_Means()
clf.fit(X_train)

correct = 0
for i in range(len(X_test)):
    predict_me = np.array(X_test[i].astype(float))
    predict_me = predict_me.reshape(-1, len(predict_me))
    prediction = clf.predict(predict_me)
    print("Predict: ", prediction, " || Actual: ", y_test[i])
    if prediction == y_test[i]:
        correct += 1


print("Accuracy",correct/len(X_test))

Predict:  3  || Actual:  Preparation
Predict:  3  || Actual:  Preparation
Predict:  0  || Actual:  Preparation
Predict:  4  || Actual:  Stroke
Predict:  3  || Actual:  Descanso
Predict:  4  || Actual:  Preparation
Predict:  3  || Actual:  Stroke
Predict:  3  || Actual:  Stroke
Predict:  1  || Actual:  Retraction
Predict:  0  || Actual:  Preparation
Predict:  3  || Actual:  Preparation
Predict:  1  || Actual:  Descanso
Predict:  3  || Actual:  Descanso
Predict:  3  || Actual:  Hold
Predict:  3  || Actual:  Stroke
Predict:  3  || Actual:  Stroke
Predict:  3  || Actual:  Preparation
Predict:  3  || Actual:  Preparation
Predict:  3  || Actual:  Stroke
Predict:  0  || Actual:  Preparation
Predict:  3  || Actual:  Preparation
Predict:  3  || Actual:  Stroke
Predict:  3  || Actual:  Descanso
Predict:  3  || Actual:  Preparation
Predict:  3  || Actual:  Stroke
Predict:  3  || Actual:  Hold
Predict:  0  || Actual:  Preparation
Predict:  3  || Actual:  Descanso
Predict:  3  || Actual:  Descanso
